In [ ]:
train = read.csv(file = '../input/457-engineered/train_engineered (2).csv', header = TRUE)
test = read.csv(file = '../input/457-engineered/test_engineered (2).csv', header = TRUE)
train_sub = train[train$Sales > 0, ]#only use positive Sales in training data
testOpen = (test$Open == 1)
test_sub = test[testOpen, ] #only needs to do prediction for open days
colnames(train_sub)

# Boosting + XGB

In [ ]:
library(gbm)
library(caret)

In [ ]:
tune_grid <- expand.grid(
  nrounds = c(800),
  eta = c(0.1),
  max_depth = 4,
  gamma = 0,
  colsample_bytree = c(1),
  min_child_weight = c(3),
  subsample = 1
)
tune_control <- caret::trainControl(
  method = "cv", # cross-validation
  number = 5, # with n folds 
  verboseIter = FALSE, # no training log
  allowParallel = TRUE # FALSE for reproducible results 
)

set.seed(123)
formula = Customers ~ factor(Store) + factor(DayOfWeek) + Promo + SchoolHoliday + factor(year) + factor(month) + day + factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):SchoolHoliday + factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + factor(month):day + CompetitionDistance + StoreType
xgb_tune_c <- train(
                formula,
                data = train_sub,
                trControl = tune_control,
                tuneGrid = tune_grid,
                method = "xgbTree",
                verbose = TRUE,
                metric="RMSE"
)
test_sub$Customers = predict(xgb_tune_c, test_sub[,-c(3)])# delete open

In [ ]:
boost.data=gbm(log(Sales) ~ ., 
                 data = train_sub[,-5], # delete open
                 distribution="gaussian", 
                 n.tree=3000,
                 interaction.depth=3,
                 cv.folds = 5)
cv.num = gbm.perf(boost.data)
yhat.boost.cv=predict(boost.data,newdata=test_sub[,-3],n.trees=cv.num)


In [ ]:
outDat = data.frame(ID = 1:nrow(test), Sales = rep(0,nrow(test))) 
outDat$Sales[testOpen] = exp(yhat.boost.cv)

#Write the prediction to an output file
write.csv(outDat, file = 'Boosting&XGB_0322.csv', row.names = FALSE)

In [ ]:
outDat